# Data Integration & Fuzzy Matching

**Objectif**: Fusionner Donnees_IA + Feedtables, fuzzy matching sur noms, annotation manuelle IID/OOD

**Pourquoi?** Les données proviennent de sources différentes, pas d'identifiant commun, les noms peuvent varier légèrement. Fuzzy matching pour trouver correspondances proches, annotation manuelle pour créer un dataset d'entraînement OOD one-shot depuis feedtables.

In [32]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
from skrub import Cleaner
import warnings
warnings.filterwarnings('ignore')
DATA_DIR = '../data/sources/'

## 1. Charger et nettoyer

In [33]:
# Charger les données
df_ia = pd.read_excel(DATA_DIR + 'Donnees_IA_2025.xlsx')
df_fb = pd.read_excel(DATA_DIR + 'feedtables_definitions_formatted.xlsx', sheet_name='Données Nutritionnelles')

# Nettoyer avec skrub
cleaner = Cleaner()
df_ia = cleaner.fit_transform(df_ia)
df_fb = cleaner.fit_transform(df_fb)

df_fb.columns = df_fb.columns.str.replace('_kg', '/kg', regex=False)
df_fb = df_fb.rename(columns={'Nom': 'Nom_fb'})

# Filtrer classes non pertinentes
df_fb = df_fb[~df_fb['Classe'].isin(['Minéraux', 'Acides aminés et autres produits', 'Huiles et corps gras'])]

# Variables d'intérêt
vars_expl = ['MS % brut', 'PB % brut', 'CB % brut', 'MGR % brut', 'MM % brut',
             'NDF % brut', 'ADF % brut', 'Lignine % brut', 'Amidon % brut', 'Sucres % brut']
vars_cibles = ['EB (kcal) kcal/kg brut', 'ED porc croissance (kcal) kcal/kg brut',
               'EM porc croissance (kcal) kcal/kg brut', 'EN porc croissance (kcal) kcal/kg brut',
               'EMAn coq (kcal) kcal/kg brut', 'EMAn poulet (kcal) kcal/kg brut',
               'UFL 2018 par kg brut', 'UFV 2018 par kg brut',
               'PDIA 2018 g/kg brut', 'PDI 2018 g/kg brut', 'BalProRu 2018 g/kg brut']

cols_keep = ['Nom'] + vars_expl + vars_cibles
df_ia = df_ia[cols_keep].drop_duplicates()

# Feedtables: garder vars expl + cibles + Nom + Definition
df_fb = df_fb.rename(columns={'Nom': 'Nom_fb',
                              'PDIA 2018 g_kg brut': 'PDIA 2018 g/kg brut',
                              'PDI 2018 g_kg brut': 'PDI 2018 g/kg brut',
                              'BalProRu 2018 g_kg brut': 'BalProRu 2018 g/kg brut'})
cols_fb = ['Nom_fb'] + vars_expl + vars_cibles + ['Definition']
df_fb = df_fb[[c for c in cols_fb if c in df_fb.columns]].drop_duplicates()

print(f" Donnees_IA: {len(df_ia)}")
print(f" Feedtables: {len(df_fb)}")

 Donnees_IA: 6352
 Feedtables: 198


## 2. Fuzzy matching

In [34]:
def fuzzy_dist(s1, s2):
    return 1 - SequenceMatcher(None, str(s1).lower(), str(s2).lower()).ratio()

ia_names = df_ia['Nom'].drop_duplicates().tolist()
fb_names = df_fb['Nom_fb'].drop_duplicates().tolist()

matches = []
for fb_name in fb_names:
    best_dist = float('inf')
    best_ia = None
    for ia_name in ia_names:
        d = fuzzy_dist(fb_name, ia_name)
        if d < best_dist:
            best_dist = d
            best_ia = ia_name
    matches.append({'Nom_IA': best_ia, 'Nom_Feedtables': fb_name, 'match_distance': best_dist})

matches_df = pd.DataFrame(matches).sort_values('match_distance')
print(f" Matches: {len(matches_df)}")
print(f" Distance min/max: {matches_df['match_distance'].min():.3f} - {matches_df['match_distance'].max():.3f}")

 Matches: 198
 Distance min/max: 0.000 - 0.600


## 3. Exporter pour annotation

In [35]:
output_file = DATA_DIR + 'fuzzy_matches_for_annotation.xlsx'
matches_df.to_excel(output_file, index=False, sheet_name='Matches')

print(f" Exporté: {output_file}")
print(f"\n À faire:")
print(f"  1. Ouvrir {output_file}")
print(f"  2. Ajouter colonne 'manual_decision' (1=même produit, 0=différent)")
print(f"  3. Sauvegarder sous: fuzzy_matches_annotated.xlsx")

 Exporté: ../data/sources/fuzzy_matches_for_annotation.xlsx

 À faire:
  1. Ouvrir ../data/sources/fuzzy_matches_for_annotation.xlsx
  2. Ajouter colonne 'manual_decision' (1=même produit, 0=différent)
  3. Sauvegarder sous: fuzzy_matches_annotated.xlsx


## 4. Charger annotations et classifier

In [36]:
# Charger annotations
anno_file = DATA_DIR + 'fuzzy_matches_annotated.xlsx'
try:
    anno_df = pd.read_excel(anno_file)
    print(f" Chargé: {anno_file}")
except FileNotFoundError:
    print(f" Fichier non trouvé: {anno_file}")
    print(f"   Exécutez d'abord l'étape 3 et annotez le fichier généré")
    raise

# Vérifier la colonne
if 'manual_decision' not in anno_df.columns:
    raise ValueError("Colonne 'manual_decision' manquante!")

same = (anno_df['manual_decision'] == 1).sum()
diff = (anno_df['manual_decision'] == 0).sum()
print(f"\n Annotations:")
print(f"  - Même produit (1): {same}")
print(f"  - Différent (0): {diff}")

 Chargé: ../data/sources/fuzzy_matches_annotated.xlsx

 Annotations:
  - Même produit (1): 76
  - Différent (0): 122


## 5. Fusionner et classifier OOD/IID

In [37]:
# Préparer IA
df_ia['source'] = 'Donnees_IA'
df_ia['OOD'] = 0
df_ia['match_distance'] = 0.0

# Enrichir IA avec descriptions Feedtables
def_map = dict(zip(df_fb['Nom_fb'], df_fb['Definition']))
match_map = dict(zip(matches_df['Nom_Feedtables'], matches_df['Nom_IA']))
df_ia['Definition'] = df_ia['Nom'].apply(
    lambda x: def_map.get(next((fb for fb in match_map.keys() if match_map[fb] == x), None))
)

# Préparer Feedtables
df_fb['source'] = 'Feedtable'
dist_map = dict(zip(matches_df['Nom_Feedtables'], matches_df['match_distance']))
df_fb['match_distance'] = df_fb['Nom_fb'].map(dist_map)
df_fb['OOD'] = (df_fb['match_distance'] >= 0.25).astype(int)  # Seuil initial
df_fb = df_fb.rename(columns={'Nom_fb': 'Nom'})

# Appliquer annotations manuelles
anno_map = dict(zip(anno_df['Nom_Feedtables'], anno_df['manual_decision']))
for idx, row in df_fb.iterrows():
    nom = row['Nom']
    if nom in anno_map and pd.notna(anno_map[nom]):
        df_fb.at[idx, 'OOD'] = 0 if anno_map[nom] == 1 else 1

# Fusionner
df_merged = pd.concat([df_ia, df_fb], ignore_index=True)

# IMPORTANT: Dédupliquer par (Nom, Definition) pour éviter les authentiques doublons IA
df_merged = df_merged.drop_duplicates(subset=['Nom', 'Definition', 'MS % brut', 'PB % brut', 'CB % brut', 'MGR % brut', 'MM % brut',
             'NDF % brut', 'ADF % brut', 'Lignine % brut', 'Amidon % brut', 'Sucres % brut'], keep='first')

iid = (df_merged['OOD'] == 0).sum()
ood = (df_merged['OOD'] == 1).sum()

print(f" Fusionnées: {len(df_merged)} produits")
print(f"  - IID (OOD=0): {iid}")
print(f"  - OOD (OOD=1): {ood}")

# Exporter
output = DATA_DIR + 'data_merged_with_ood_classification.xlsx'
df_merged.to_excel(output, index=False)
print(f"\n Exporté: {output}")

 Fusionnées: 6550 produits
  - IID (OOD=0): 6428
  - OOD (OOD=1): 122

 Exporté: ../data/sources/data_merged_with_ood_classification.xlsx
